In [1]:
import math
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 2500)

# Loading historical data
hs_param = pd.read_csv('C:/Users/preety.tiwari/Downloads/Fw__data_set/historical parameters.csv')
# Setting up the parameters differently
decay_factor = hs_param.decay_factor.iloc[0]
holding_period = hs_param.holding_period.iloc[0]
# print(holding_period)
confidence_level = hs_param.confidence_level.iloc[0]

rates_E = pd.read_csv('C:/Users/preety.tiwari/Downloads/Fw__data_set/rates_instrument.csv')
rates_E = rates_E[rates_E["instrument_type"] == 'E']
rates_E.instrument = rates_E.instrument.astype('str')
rates_E["key"] = "instru" + '_'+rates_E.instrument

# Simulated Instrument rates
simulated_instruments = pd.DataFrame([])

for a, h in rates_E.groupby('instrument'):
    g = h.filter(["instrument", "instrument_type", "value", "key"])
    new_row = g.iloc[0].to_dict()
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    g = g.reset_index(drop = True)
    g['simulated_rates'] = np.log(g["value"]/g["value"].shift(-holding_period, fill_value = g["value"][0]))
    
    g = pd.concat([top_row, g]).reset_index(drop = True)

    g['new_value'] = g['simulated_rates'].apply(lambda x: np.exp(x)*g["value"].iloc[0])
    simulated_instruments = simulated_instruments.append(g)



rates_exchange = pd.read_csv("C:/Users/preety.tiwari/Downloads/Fw__data_set/rates_exchange.csv")

if len(rates_exchange) != 0:
    rates_exchange["key"] = rates_exchange["currency_from"] + "_"+rates_exchange["currency_to"]

simulated_exchange = pd.DataFrame([])
for i,k in rates_exchange.groupby('key'):
    j = k.filter(["currency_from", "currency_to", "rate", "key"])
    new_row = j.iloc[0].to_dict()
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    
    j = j.reset_index(drop = True)
    j['simulated_rates'] = np.log(j['rate']/j['rate'].shift(-holding_period, fill_value = j["rate"][0]))
    j = pd.concat([top_row, j]).reset_index(drop = True)
    j["new_rate"] = j['simulated_rates'].apply(lambda x: np.exp(x)*j["rate"].iloc[0])
    simulated_exchange = simulated_exchange.append(j)

simulated_exchange.head(5)

tx_eq = pd.read_csv('C:/Users/mitesh.shingare/Downloads/Fw__data_set/transaction_equity.csv')
n=hs_param['no_of_days'].iloc[0]

def get_mtm_bc(idx,tx_df, ex_df, instr_df):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        h["keyMatch"] = h['currency'] + '_' + h['currency_base']
        keyMatch = h["keyMatch"].iloc[0]
        instrMatch = h["instrument"].iloc[0]
        unitCount = h['unit_count'].iloc[0]
        
        crss_rate_df = ex_df[ex_df['key']==keyMatch]
        cross_rate = crss_rate_df['new_rate'].iloc[0]
        
        equity_rate_df = instr_df[instr_df['instrument'] == instrMatch]
        equity_rate = equity_rate_df['new_value'].iloc[0]
        
        mtm_bc = cross_rate * equity_rate * unitCount
        result = {'idx': idx, "env":k,"instrument": instrMatch, "unit_count":unitCount, "accounting_book":h["accounting_book"].iloc[0],
                 "accounting_portfolio": h["accounting_portfolio"].iloc[0], "currency":h["currency"].iloc[0], "mtm_bc":mtm_bc}
        res_df.append(result)

    return res_df

final_df = pd.DataFrame()
for i in range(0, n):
    exch_rates = simulated_exchange[simulated_exchange.index == i]
    instr_rates = simulated_instruments[simulated_instruments.index == i]
    result = get_mtm_bc(i, tx_eq, exch_rates, instr_rates)
    res = pd.DataFrame(result)

    final_df = final_df.append(res)



final_df.to_csv("final_result.csv",index = False)
simulated_exchange.to_csv("simulated_exchange.csv", index=False)
simulated_instruments.to_csv("simulated_instrument.csv", index = False)

C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1402172126.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1402172126.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1402172126.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1402172126.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use panda

In [ ]:
final_df.to_csv("C:/Users/mitesh.shingare/Downloads/Fw__data_set/final_result.csv",index = False)

In [ ]:
final_df

In [191]:
#df.sort_values("city08")
final_df = final_df.sort_values(by =["env","idx"], ascending = [True, True])
final_df

,idx,env,instrument,unit_count,accounting_book,accounting_portfolio,currency,mtm_bc
0,0,HDFCbank1942,HDFCbank,11000,1,1,INR,1.501555e+07
0,1,HDFCbank1942,HDFCbank,11000,1,1,INR,1.494766e+07
0,2,HDFCbank1942,HDFCbank,11000,1,1,INR,1.496504e+07
0,3,HDFCbank1942,HDFCbank,11000,1,1,INR,1.492583e+07
0,4,HDFCbank1942,HDFCbank,11000,1,1,INR,1.510761e+07
0,5,HDFCbank1942,HDFCbank,11000,1,1,INR,1.504952e+07
0,6,HDFCbank1942,HDFCbank,11000,1,1,INR,1.517085e+07
0,7,HDFCbank1942,HDFCbank,11000,1,1,INR,1.552763e+07
0,8,HDFCbank1942,HDFCbank,11000,1,1,INR,1.556801e+07
0,9,HDFCbank1942,HDFCbank,11000,1,1,INR,1.549581e+07


In [192]:
change = final_df['mtm_bc'].iloc[0]

In [193]:
def change(final_df):
    
    change_df = pd.DataFrame()
    
    for i,j in final_df.groupby('instrument'):
        curr_val = j['mtm_bc'].iloc[0]
        j['change_in_mtm'] = j['mtm_bc'].apply(lambda x: x - curr_val)
        
        change_df = change_df.append(j)
        
    return change_df
        

In [194]:
change_df= change(final_df)

C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
C:\Users\mitesh.shingare\AppData\Local\Temp\ipykernel_75108\1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
C:\Users\mitesh.shingare

In [195]:
len(change_df)

1250

In [196]:
total_change_in_mtm = change_df.groupby('idx')[['change_in_mtm']].sum()

In [197]:
total_change_in_mtm 

,change_in_mtm
idx,
0,0.000000e+00
1,7.234927e+05
2,4.346984e+04
3,-2.222029e+05
4,-1.242359e+06
5,-1.838257e+06
6,-1.397515e+06
7,-1.816423e+06
8,-3.695579e+05


In [198]:
len(total_change_in_mtm)

250

In [199]:
total_change_in_mtm = total_change_in_mtm['change_in_mtm'].tolist()

In [200]:
total_change_in_mtm

[0.0,
 723492.7057282273,
 43469.84394807741,
 -222202.8721384462,
 -1242359.0992399845,
 -1838256.520457767,
 -1397514.7791441195,
 -1816423.2091310602,
 -369557.85867754,
 2242865.5736109763,
 2578092.814062981,
 3669998.0159030445,
 1456143.8393597063,
 3530944.0174306296,
 4965581.258452943,
 3965102.5162277278,
 2540307.227322327,
 2758838.3685336784,
 2132704.932634363,
 -1357267.6802142374,
 -3707123.193025209,
 -4104251.5960600413,
 -3559955.302972669,
 -7049862.028997101,
 -8600640.00382183,
 -7318317.438586464,
 -4709996.225558905,
 -4678404.718925968,
 -4913302.193787793,
 448259.27139666304,
 3918547.402838886,
 3883109.2927430924,
 2223613.4231771007,
 2870147.440048171,
 3629596.599547945,
 5524698.855063921,
 357503.7039486822,
 1412140.874911055,
 4245365.350590419,
 1294150.1680713054,
 358017.2494835295,
 -1832678.430527702,
 -1100784.5959402882,
 310424.124368906,
 -787796.9595412649,
 -5075694.555072518,
 -887774.5229377896,
 -3301558.7324363347,
 -5513655.643064536

In [201]:
if (decay_factor == 0):
    VaR_exact_equity_hp = -(np.quantile(total_change_in_mtm, 0.01))
    total_change_in_mtm_sorted = sorted(total_change_in_mtm, reverse = True)
    nearby = np.floor(confidence_level * len(total_change_in_mtm))
    nearby = int(nearby)
    nearby_nxt = nearby + 1

In [202]:
diff1 = pd.to_numeric(VaR_exact_equity_hp + total_change_in_mtm_sorted[nearby])
diff2 = pd.to_numeric(- total_change_in_mtm_sorted[nearby_nxt] - VaR_exact_equity_hp)

In [203]:
if (diff1 == diff2):
    VaR_nearby_equity_hp = - total_change_in_mtm_sorted[nearby + 1]
elif (diff1 < diff2):
    VaR_nearby_equity_hp = -total_change_in_mtm_sorted[nearby]
else:
    VaR_nearby_equity_hp = - total_change_in_mtm_sorted[nearby + 1]

In [204]:
VaR_nearby_equity_hp

7489518.753564825

In [205]:
# decay factor

In [206]:
if (decay_factor > 0 and decay_factor < 1):
    decay_weights = ((1-decay_factor)* c(decay_factor^(0:(length(unique(result$env))-2)))/(1-decay_factor^((length(unique(result$env))-1))))


SyntaxError: invalid syntax (2616512139.py, line 2)

In [207]:
print(result)

[{'idx': 249, 'env': 'HDFCbank1942', 'instrument': 'HDFCbank', 'unit_count': 11000, 'accounting_book': 1, 'accounting_portfolio': 1, 'currency': 'INR', 'mtm_bc': 15935700.0}, {'idx': 249, 'env': 'Hindustanlever914', 'instrument': 'HLL', 'unit_count': 7500, 'accounting_book': 1, 'accounting_portfolio': 1, 'currency': 'INR', 'mtm_bc': 17836125.0}, {'idx': 249, 'env': 'Infosys203', 'instrument': 'Infosys', 'unit_count': 13550, 'accounting_book': 1, 'accounting_portfolio': 1, 'currency': 'INR', 'mtm_bc': 21547209.999999996}, {'idx': 249, 'env': 'Reliance1908', 'instrument': 'Reliance', 'unit_count': 6000, 'accounting_book': 1, 'accounting_portfolio': 1, 'currency': 'INR', 'mtm_bc': 12724199.999999998}, {'idx': 249, 'env': 'TCS108', 'instrument': 'TCS', 'unit_count': 10000, 'accounting_book': 1, 'accounting_portfolio': 1, 'currency': 'INR', 'mtm_bc': 32163500.000000004}]


In [208]:
#((1-decay_factor)* c(decay_factor^(0:(length(unique(result$env))-2)))/(1-decay_factor^((length(unique(result$env))-1))))
#((1 - decay_factor) * (decay_factor )math.pow((0:(len(change_df['idx'].unique())-2)) / (1 - decay_factor ^ (len(change_df['idx'])))))

In [209]:
5**2

25

In [210]:
#1- ref
#decay1 = decay_factor ** range(0,(len(change_df['idx'].unique())-1))

In [211]:
decay1 = decay_factor ** range(0,(len(change_df['idx'].unique())-1))

In [212]:
decay1.tolist()

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [213]:
len(decay1)

249

In [214]:
#2
decay2 = (1 - decay_factor) ** (len(change_df['idx'].unique() -1))

In [215]:
decay3 =decay1 / decay2

In [216]:
decay3

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [217]:
decay_weights = (1 - decay_factor) * decay3

In [218]:
decay_weights

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [219]:
total_mtm = change_df.groupby('idx')[['mtm_bc']].sum()
total_mtm

,mtm_bc
idx,
0,1.017476e+08
1,1.024711e+08
2,1.017911e+08
3,1.015254e+08
4,1.005053e+08
5,9.990936e+07
6,1.003501e+08
7,9.993119e+07
8,1.013781e+08


In [220]:
total_mtm['mtm_bc'][0]

101747615.0

In [221]:
mtm_returns = []

for i in range(1,(len(change_df['idx'].unique() -1))):
    fv = total_mtm['mtm_bc'][0]
    returns = (total_mtm['mtm_bc'][i] - fv)/fv
    mtm_returns.append(returns)                                    

In [222]:
(len(change_df['idx'].unique() -1))

250

In [223]:
#mtm1 <- mtm[order(mtm_returns),]
mtm1 = mtm_returns.sort()

In [224]:
decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])

In [225]:
mtm = pd.concat([mtm_returns,decay_weights], axis = 1)

In [226]:
mtm

,mtm_returns,decay_weights
0,-0.084529,1.0
1,-0.081599,0.0
2,-0.073609,0.0
3,-0.071926,0.0
4,-0.069878,0.0
5,-0.069288,0.0
6,-0.068237,0.0
7,-0.067246,0.0
8,-0.066338,0.0
9,-0.066179,0.0


In [230]:
mtm = mtm.sort_values(['mtm_returns'])

In [228]:
mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])

In [229]:
mtm

,mtm_returns,decay_weights,cum_decay_weights
0,-0.084529,1.0,1.0
1,-0.081599,0.0,1.0
2,-0.073609,0.0,1.0
3,-0.071926,0.0,1.0
4,-0.069878,0.0,1.0
5,-0.069288,0.0,1.0
6,-0.068237,0.0,1.0
7,-0.067246,0.0,1.0
8,-0.066338,0.0,1.0
9,-0.066179,0.0,1.0


In [236]:
a = 1-confidence_level
mtm1 = mtm.loc[mtm['cum_decay_weights'] < a]

In [237]:
mtm1

,mtm_returns,decay_weights,cum_decay_weights


In [238]:
a

0.010000000000000009

In [242]:
total_mtm

,mtm_bc
idx,
0,1.017476e+08
1,1.024711e+08
2,1.017911e+08
3,1.015254e+08
4,1.005053e+08
5,9.990936e+07
6,1.003501e+08
7,9.993119e+07
8,1.013781e+08


In [243]:
if (len(mtm1)==0):
    VaR_exact_equity_hp = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])

    VaR_nearby_equity_hp = VaR_exact_equity_hp

else:
    index = len(mtm1['cum_decay_weights'])

In [148]:
len(decay_weights)

249

In [144]:
len(mtm_returns)

250

In [158]:
mtm = pd.DataFrame(mtm_returns , decay_weights)
mtm

,0
1.0,0.000000
0.0,0.007111
0.0,0.000427
0.0,-0.002184
0.0,-0.012210
0.0,-0.018067
0.0,-0.013735
0.0,-0.017852
0.0,-0.003632
0.0,0.022043


In [ ]:
##total_change_in_mtm = change_df.groupby('idx')[['change_in_mtm']].sum()

In [91]:
a=2
b=3
c=math.pow(a,b)
c

8.0

In [46]:
total_change_in_mtm[5]

6832080.152352195

In [63]:
#newList.sort()
#return newList
#answer = sorted(newList)

total_change_in_mtm_sorted = sorted(total_change_in_mtm, reverse = True)

In [54]:
##total_change_in_mtm_sorted = total_change_in_mtm.sort(reverse = False)

In [56]:
print(total_change_in_mtm_sorted)

None


In [ ]:
change()

In [ ]:
change_in_mtm_bc

In [ ]:
#pd.DataFrame(change_in_mtm_bc)

In [ ]:
type(change)

In [ ]:
#change_in_mtm_bc=pd.DataFrame(change_in_mtm_bc)
#df3 = pd.concat([df1, df2], ignore_index=True).reindex(df1.columns, axis='columns')
#df3=pd.concat([final_df,change], ignore_index=True).reindex(change.columns,axis='columns')

In [ ]:
 #final_df['change_in_mtm']

In [ ]:
#final_df

In [ ]:
total_change_in_mtm <-tapply(result$change_in_mtm_bc, result$env, sum)
total_change_in_mtm = total_change_in_mtm[2:length(total_change_in_mtm)]

# total_change_in_mtm = tapply(final_df['change_in_mtm_bc'],result['env'])